# Discrete derivatives and Prophet 

The purpose of this notebook is to set up the use of the first derivative of the data as a regressor in Prophet. Since the first derivative is needed as a future regressor, we run the Prophet first without regressors, obtain a list of forecasted values, use those to compute future derivatives, and then use those as a future regressor. 

In [1]:
pip install prophet

Note: you may need to restart the kernel to use updated packages.


# Imports 

In [11]:
import pandas as pd
import numpy as np 
import matplotlib 
import matplotlib.pyplot as plt
from sklearn import metrics
from prophet import Prophet
from prophet.diagnostics import cross_validation
from prophet.diagnostics import performance_metrics
import pickle
import sys
import os

# Derivative

In [2]:
def derivative_new(my_list):
    return [my_list[i]-my_list[i-1] for i in range(1,len(my_list))]

In [3]:
def kthderiv(extended_list, k, original_length):
    '''
    Parameters:
        extended_list (list): list to take derivatives of plus at least k predictions
        k (int): number of derivatives
        original_length (int): length of original list
    
    Returns:
        derivatives (numpy array): the ith column has the ith derivative of the list, up to the kth derivative
    '''
    
    derivatives = extended_list[:original_length]
    for i in range(1,k+1):
        next_deriv = extended_list[:original_length + i]
        for j in range(i):
            next_deriv = derivative_new(next_deriv)
        derivatives = np.column_stack((derivatives, next_deriv))
    return derivatives

# Prophet with Derivatives and Regressors

In [4]:
def create_df(x, y):
    df = pd.DataFrame()
    df['ds'] = pd.to_datetime(x, unit='D')
    df['y'] = y
    return df

In [5]:
def holdout_variance_explained(pred, pred_length, y_pred):
    '''
    Parameters:
        pred (list): list of predictions
        pred_length (int): length of prediction
        y_pred (list): correct future values
        
    Returns:
        Percentage of variance explained for holdout
    '''
    y_pred = np.array(y_pred)
    score = np.sqrt(metrics.mean_squared_error(pred[-pred_length:],y_pred[-pred_length:]))
    variance_explained = ((y_pred[-pred_length:].std()-score)/y_pred[-pred_length:].std())*100
    return variance_explained

In [6]:
def prophetpredict(training,horizon,horizon_freq,regressors,retail_data = True):
    '''
    Parameters:
        training (dataframe): ['ds'] contains dates, ['y'] contains values
        horizon (int): number of predictions
        horizon_freq (string): time interval between predictions
        regressors (dataframe): all values for regressors, including future regressors
        retail_data (bool): set to True when running on real data
    
    Returns:
        List of model outputs for known dates plus predictions
    '''
    data_length = len(training.index)
    m = Prophet(yearly_seasonality=True, daily_seasonality=False, weekly_seasonality=False)
    if retail_data:
        m.add_country_holidays(country_name='US')
        m.train_holiday_names
    
    for col in regressors:
        m.add_regressor(col)
    all_input = pd.concat((training, regressors.head(data_length)), axis=1)
    m.fit(all_input)
    future = m.make_future_dataframe(periods=horizon, freq=horizon_freq)
    all_future_input = pd.concat((future, regressors.head(data_length+horizon)), axis=1)
    forecast = m.predict(all_future_input)
    predicteddf=forecast[['ds', 'yhat']]
    return list(predicteddf['yhat'])

In [7]:
def prophetnderivatives(df, horizon, horizon_freq, n_derivs, regressors, retail_data=True):
    '''
    Parameters:
        df (dataframe): ['ds'] contains all dates, ['y'] contains all values (train and test combined)
        horizon (int): length of test set/number of predictions
        horizon_freq (string): time interval between predictions
        n_derivs (int): number of derivative regressors to use
        regressors (dataframe): all values for regressors
        retail_data (bool): set to True when running on real data
        
    Returns:
        Percentage of variance explained for holdout
    '''
    data_length = len(df.index) - horizon
    
    if n_derivs == 0:
        pred = prophetpredict(df.head(data_length), horizon + n_derivs, horizon_freq, regressors, retail_data)
        return holdout_variance_explained(pred, horizon, list(df['y']))
        
    pred = prophetpredict(df.head(data_length), horizon + n_derivs, horizon_freq, pd.DataFrame(), retail_data)
    
    y_train = list(df['y'].head(data_length))
    
    col_names = ['derivative_' + str(i+1) for i in range(n_derivs)]
    future_derivatives = kthderiv(y_train + pred[data_length:], n_derivs, data_length + horizon)[:,1:]
    future_derivatives = pd.DataFrame(data=future_derivatives, columns=col_names)
    all_future_regressors = pd.concat((future_derivatives, regressors), axis=1)

    P = prophetpredict(df.head(data_length), horizon, horizon_freq, all_future_regressors, retail_data)
    variance_explained = holdout_variance_explained(P, horizon, list(df['y']))
    return variance_explained

# Sales Data Example

In [8]:
salesdata = pd.read_excel('All_Data_Formatted.xlsx', header=1)
salesdata = salesdata.rename({'date':'ds'}, axis=1)
salesdata = salesdata.iloc[84:204, :]
ds = salesdata['ds']
sales = salesdata.iloc[:, 1:13]
indicators = salesdata.iloc[:, 13:]
indicators.reset_index(inplace=True, drop=True)
sales = pd.concat((ds, sales), axis=1)
sales.reset_index(inplace=True, drop=True)

In [9]:
print(sales)
print(indicators)

            ds   RSAFS  RSFSXMV  RSHPCS  RSGASS  RSCCAS  RSSGHBMS  RSGMS  \
0     2010/1/1  346043   289245   21187   37399   17351      6507  49844   
1     2010/2/1  346835   291032   21199   36950   17402      6412  50538   
2     2010/3/1  354360   293463   21363   37123   17908      6515  50095   
3     2010/4/1  357461   295638   21471   37218   17724      6409  50650   
4     2010/5/1  354255   292684   21399   36606   17656      6496  50023   
..         ...     ...      ...     ...     ...     ...       ...    ...   
115   2019/8/1  521158   415426   28580   42735   22615      6740  60074   
116   2019/9/1  517755   414270   28600   42589   22376      6698  59377   
117  2019/10/1  519519   416179   28809   43254   22417      6742  59662   
118  2019/11/1  521774   416055   28576   43246   22184      6532  60033   
119  2019/12/1  525903   422153   28267   44459   23230      6720  60632   

     RSDSELD  RSMSR  RSNSR  RSFSDP   RSXFS  
0      15585   8363  27236   37700  308343

In [10]:
prophet_results = {}
horizon = len(sales.index)//10
for sale in sales.iloc[:,1:]:
    data = sales[['ds', sale]]
    data = data.rename(columns={sale: 'y'})
    for deriv in range(5):
        result = prophetnderivatives(data, horizon, 'm', deriv, pd.DataFrame())
        print(sale, deriv, result)
        prophet_results[sale, '', deriv] = result
        for indicator in indicators:
            result = prophetnderivatives(data, horizon, 'm', deriv, pd.DataFrame(indicators[indicator]))
            print(sale, indicator, deriv, result)
            prophet_results[sale, indicator, deriv] = result

10:43:21 - cmdstanpy - INFO - Chain [1] start processing
10:43:21 - cmdstanpy - INFO - Chain [1] done processing
10:43:24 - cmdstanpy - INFO - Chain [1] start processing


RSAFS 0 15.324039282504836


10:43:24 - cmdstanpy - INFO - Chain [1] done processing
10:43:27 - cmdstanpy - INFO - Chain [1] start processing
10:43:27 - cmdstanpy - INFO - Chain [1] done processing


RSAFS break_inflate10 0 48.79709787286957


10:43:29 - cmdstanpy - INFO - Chain [1] start processing
10:43:29 - cmdstanpy - INFO - Chain [1] done processing


...

RSXFS International_Trade 4 -121.22640117094494


11:17:36 - cmdstanpy - INFO - Chain [1] start processing
11:17:36 - cmdstanpy - INFO - Chain [1] done processing
11:17:39 - cmdstanpy - INFO - Chain [1] start processing
11:17:39 - cmdstanpy - INFO - Chain [1] done processing


RSXFS Med_Cpi 4 -80.22809461589964


11:17:41 - cmdstanpy - INFO - Chain [1] start processing
11:17:41 - cmdstanpy - INFO - Chain [1] done processing


RSXFS CPI 4 -102.86020359614379


In [12]:
#with open('prophet_results.pickle', 'wb') as handle:
#    pickle.dump(prophet_results, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [20]:
# make sure prophet_results.pickle is in the same folder as this Jupyter notebook
with open('prophet_results.pickle', 'rb') as handle:
    a = pickle.load(handle)

In [21]:
def dumpclean(obj):
    if isinstance(obj, dict):
        for k, v in obj.items():
            if hasattr(v, '__iter__'):
                print(k)
                dumpclean(v)
            else:
                print('%s : %s' % (k, v))
    elif isinstance(obj, list):
        for v in obj:
            if hasattr(v, '__iter__'):
                dumpclean(v)
            else:
                print(v)
    else:
        print(obj)

In [28]:
sale_names = []
indicator_names = []
derivative_numbers = []
variance_values = []
for key, value in a.items():
    sale_names.append(key[0])
    indicator_names.append(key[1])
    derivative_numbers.append(key[2])
    variance_values.append(value)

In [37]:
sheet = np.column_stack((sale_names, indicator_names, derivative_numbers, variance_values))
sheet = pd.DataFrame(sheet, columns=['Sale', 'Indicator', 'Derivatives', 'Variance explained'])
sheet.to_csv('prophet_economic_indicators.csv')

In [56]:
results_by_sale = {}
for key, value in a.items():
    results_by_sale[key[0]] = {}
    
for key, value in a.items():
    results_by_sale[key[0]][key] = value

In [57]:
best_results = {}
for sale in results_by_sale:
    best_results[sale] = {}
for sale, dictionary in results_by_sale.items():
    best_results[sale][sale,'',0] = dictionary[sale,'',0]
    best_results[sale][max(dictionary, key=dictionary.get)] = dictionary[max(dictionary, key=dictionary.get)]

In [59]:
dumpclean(best_results)

RSAFS
('RSAFS', '', 0) : 15.324039282504836
('RSAFS', 'break_inflate10', 0) : 48.79709787286957
RSFSXMV
('RSFSXMV', '', 0) : -7.847762127188558
('RSFSXMV', 'break_inflate10', 0) : 39.98704377614626
RSHPCS
('RSHPCS', '', 0) : -356.0426699461252
('RSHPCS', 'Med_Cpi', 0) : -352.70646901936715
RSGASS
('RSGASS', '', 0) : -301.1200619804295
('RSGASS', 'break_inflate10', 2) : 13.695670087881437
RSCCAS
('RSCCAS', '', 0) : -77.49786990564135
('RSCCAS', 'indust_prod', 0) : -69.39353081408566
RSSGHBMS
('RSSGHBMS', '', 0) : -196.77899352672932
('RSSGHBMS', 'mortgage_rate30', 3) : -96.01921379894473
RSGMS
('RSGMS', '', 0) : -120.45853365128328
('RSGMS', 'mortgage_rate30', 0) : -96.23457744007699
RSDSELD
('RSDSELD', '', 0) : -94.13121498999443
('RSDSELD', 'indust_prod', 1) : -64.1288954513712
RSMSR
('RSMSR', '', 0) : -23.910841315922905
('RSMSR', 'CPI', 0) : -18.121982839483145
RSNSR
('RSNSR', '', 0) : 30.557234909407484
('RSNSR', 'mortgage_rate30', 0) : 37.50200155151784
RSFSDP
('RSFSDP', '', 0) : 